# 1. Presets 

## 1.1 Importing all necessary libraries

In [1]:
import numpy as np

# QUBO / optimization
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import to_ising

# VQE for the QUBO
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms import SamplingVQE, VQE
from qiskit_algorithms.optimizers import SPSA, SLSQP
from qiskit.primitives import StatevectorSampler
from qiskit.primitives import StatevectorEstimator

# VQE for chemistry (H2) via Qiskit Nature
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# DFT (PySCF)
from pyscf import gto, dft as pyscf_dft

## 1.2 Inserting all generation parameters

In [2]:
NUM_SITES = 4
TARGET_LONG = 2

# 2. Generating candidates with correct topological properties

## 2.1 Encode a toy MTV material into a QUBO

In [3]:
def build_mtv_qubo(num_sites=4, target_long=2):
    """
    Toy MTV model:
      - num_sites linker sites in a ring
      - x_i = 0 -> short linker (THQ-like)
      - x_i = 1 -> long linker (HHTP-like)
      - constraint: exactly `target_long` long linkers
      - objective: favor alternation 0101/1010 around the ring

    We encode:
      ratio constraint: sum_i x_i == target_long
      balance objective: sum_edges (x_i + x_j - 1)^2  (min when x_i != x_j)
    """
    qp = QuadraticProgram("mtv_ring")

    # Binary vars x_0 ... x_{N-1}
    vars_x = [qp.binary_var(f"x_{i}") for i in range(num_sites)]

    # Ratio constraint: exactly target_long "long" linkers
    qp.linear_constraint(
        linear={v.name: 1.0 for v in vars_x},
        sense="==",
        rhs=target_long,
        name="ratio_long",
    )

    # Ring edges: i -- (i+1) mod N
    edges = [(i, (i + 1) % num_sites) for i in range(num_sites)]

    # Objective: sum_edges (x_i + x_j - 1)^2
    # Expand (x_i + x_j - 1)^2 = 1 - x_i - x_j + 2 x_i x_j
    linear = {v.name: 0.0 for v in vars_x}
    quadratic = {}

    for i, j in edges:
        vi, vj = vars_x[i].name, vars_x[j].name
        linear[vi] += -1.0
        linear[vj] += -1.0
        key = (vi, vj)
        quadratic[key] = quadratic.get(key, 0.0) + 2.0
        # constant "+1" is dropped; it just shifts all energies by a constant

    qp.minimize(linear=linear, quadratic=quadratic)

    # Convert to QUBO and then to Ising form
    to_qubo = QuadraticProgramToQubo()
    qubo = to_qubo.convert(qp)
    ising_op, offset = to_ising(qubo)  # SparsePauliOp + constant shift

    return qp, qubo, ising_op, offset


## 2.2 Solve the QUBO with VQE (SamplingVQE + StatevectorSampler)

In [4]:
def solve_qubo_with_vqe(ising_op):
    """
    Use SamplingVQE to approximate the minimum eigenstate of the
    diagonal Ising Hamiltonian obtained from the QUBO.
    """
    num_qubits = ising_op.num_qubits

    sampler = StatevectorSampler(seed=1234)

    ansatz = TwoLocal(
        num_qubits,
        rotation_blocks="ry",
        entanglement_blocks="cz",
        entanglement="linear",
        reps=1,
    )
    optimizer = SPSA(maxiter=150)

    vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=optimizer)
    result = vqe.compute_minimum_eigenvalue(ising_op)

    return result


def decode_mtv_config_from_bitstring(bitstring, num_sites):
    """
    Qiskit bitstrings label qubit-0 as the rightmost bit.
    We'll reverse it so index 0 corresponds to site 0.
    """
    bits = bitstring[::-1]
    bits = bits[:num_sites]
    cfg = [int(b) for b in bits]
    return cfg  # list length = num_sites, entries 0/1

# 3. Searching for the ground state energy

## 3.1 Using DFT (classical approach)

In [5]:
def h2_bond_from_mtv_config(cfg, r0=0.74, alpha=0.30):
    """
    Very toy mapping from MTV pattern to an H-H bond length:

      - cfg: list of bits (0=short linker, 1=long linker)
      - fraction_long = (#1s) / N
      - r = r0 + alpha * (fraction_long - 0.5)

    So more long linkers -> slightly larger bond length.
    """
    N = len(cfg)
    frac_long = sum(cfg) / N
    r = r0 + alpha * (frac_long - 0.5)
    return r


def dft_energy_h2(r):
    """
    KS-DFT ground-state energy of H2 at distance r (Å)
    using PySCF (B3LYP/sto-3g).
    """
    mol = gto.Mole()
    mol.atom = f"H 0.0 0.0 0.0; H 0.0 0.0 {r}"
    mol.basis = "sto-3g"
    mol.unit = "Angstrom"
    mol.build()

    mf = pyscf_dft.RKS(mol)
    mf.xc = "b3lyp"
    e_tot = mf.kernel()
    return e_tot  # Hartree


## 3.2 Using VQE (quantum approach)

In [6]:
def vqe_energy_h2(r):
    """
    VQE ground-state energy of H2 at distance r (Å),
    using Qiskit Nature + PySCFDriver + UCCSD ansatz.

    This solves the *electronic* Hamiltonian in the chosen basis
    (sto-3g here), then adds nuclear repulsion to give the total energy.
    """
    # 1) Build electronic structure problem from PySCF
    driver = PySCFDriver(atom=f"H 0 0 0; H 0 0 {r}", basis="sto-3g")
    problem = driver.run()

    # 2) Choose qubit mapper (Jordan-Wigner)
    mapper = JordanWignerMapper()

    # 3) Build UCCSD ansatz with Hartree-Fock initial state
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # 4) VQE with Estimator primitive (no Sampler import!)
    estimator = StatevectorEstimator()
    optimizer = SLSQP(maxiter=200)
    vqe = VQE(estimator, ansatz, optimizer)
    vqe.initial_point = np.zeros(ansatz.num_parameters)

    # 5) Wrap in a GroundStateEigensolver
    solver = GroundStateEigensolver(mapper, vqe)
    result = solver.solve(problem)

    # Total ground-state energy (electronic + nuclear) in Hartree:
    return result.total_energies[0]

# 4. Full pipeline

In [7]:
# --- (a) QUBO encoding ---
qp, qubo, ising_op, offset = build_mtv_qubo(num_sites=NUM_SITES, target_long=TARGET_LONG)

print("QuadraticProgram:")
print(qp.prettyprint())
print("\nIsing operator has", ising_op.num_qubits, "qubits")

# --- (b) Solve QUBO with VQE ---
vqe_result_qubo = solve_qubo_with_vqe(ising_op)
print("\n[QUBO-VQE] approximate minimum eigenvalue (incl. offset):",
        vqe_result_qubo.eigenvalue.real + offset)

# Extract the most probable bitstring
dist = vqe_result_qubo.eigenstate  # dict {bitstring: probability}
sorted_states = sorted(dist.items(), key=lambda kv: kv[1], reverse=True)
print("\nTop QUBO bitstrings (SamplingVQE result):")
for bs, p in sorted_states[:5]:
    print(f"  {bs}  p={p:.3f}")

best_bitstring, best_prob = sorted_states[0]
best_cfg = decode_mtv_config_from_bitstring(best_bitstring, num_sites=4)
print("\nBest MTV configuration (site -> 0 short, 1 long):", best_cfg)

# --- (c) DFT energies for this configuration ---
r_h2 = h2_bond_from_mtv_config(best_cfg)
e_dft = dft_energy_h2(r_h2)
print(f"\n[DFT] H2 at r = {r_h2:.3f} Å -> E_DFT = {e_dft:.6f} Hartree")

# --- (d) VQE energies for the same geometry ---
e_vqe = vqe_energy_h2(r_h2)
print(f"[VQE] H2 at r = {r_h2:.3f} Å -> E_VQE = {e_vqe:.6f} Hartree")

print("\nDone.")


QuadraticProgram:
Problem name: mtv_ring

Minimize
  2*x_0*x_1 + 2*x_0*x_3 + 2*x_1*x_2 + 2*x_2*x_3 - 2*x_0 - 2*x_1 - 2*x_2 - 2*x_3

Subject to
  Linear constraints (1)
    x_0 + x_1 + x_2 + x_3 == 2  'ratio_long'

  Binary variables (4)
    x_0 x_1 x_2 x_3


Ising operator has 4 qubits


/var/folders/4n/x9n_6c9x73bggg4pc3n0y16w0000gn/T/ipykernel_81046/3209400767.py:10: DeprecationWarning: The class ``qiskit.circuit.library.n_local.two_local.TwoLocal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.n_local instead.
  ansatz = TwoLocal(



[QUBO-VQE] approximate minimum eigenvalue (incl. offset): -2.9560546875

Top QUBO bitstrings (SamplingVQE result):
  0110  p=0.513
  1010  p=0.486
  0100  p=0.001

Best MTV configuration (site -> 0 short, 1 long): [0, 1, 1, 0]
converged SCF energy = -1.16541841052621

[DFT] H2 at r = 0.740 Å -> E_DFT = -1.165418 Hartree


/Users/dmitrijziganov/miniforge3/envs/vqe/lib/python3.12/site-packages/pyscf/dft/libxc.py:511: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '
/Users/dmitrijziganov/miniforge3/envs/vqe/lib/python3.12/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/dmitrijziganov/miniforge3/envs/vqe/lib/python3.12/site-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


[VQE] H2 at r = 0.740 Å -> E_VQE = -1.137284 Hartree

Done.
